In [2]:
!pip install keras_tuner

     |████████████████████████████████| 71kB 7.1MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=d5f3738f67c26dc879bbf5014d128019d70ca497824ddafceccc98bacb295d6a
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=6cf153c9bffdfb2dd7284f1476f83ec40b2496642dce62fd9c20f6b863599746
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [10]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import kerastuner as kt
import IPython

In [4]:
def preprocess_data(X, Y):
    X = tf.keras.applications.densenet.preprocess_input(X)
    return X, Y

In [5]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.cifar10.load_data()
train_x, train_y = preprocess_data(train_x, train_y)
test_x, test_y = preprocess_data(test_x, test_y)

170500096/170498071 [==============================] - 2s 0us/step


In [6]:
def get_modified_inception_net(hp):
    # Tuner params
    image_size = hp.Int('image_size', min_value=75, max_value=400, step=150)
    train_layer_limit = hp.Choice('trainable_layer', values=[0, 5, 10])
    learning_rate = hp.Choice('learning_rate', values=[1e-4, 1e-5, 1e-6])
    # Model
    inception_net = tf.keras.applications.inception_v3.InceptionV3(
        input_shape=(image_size, image_size, 3),
        include_top=False,
        weights='imagenet'
    )
    if train_layer_limit == 0:
        for layer in inception_net.layers:
            layer.trainable = False
    else:
        for layer in inception_net.layers[train_layer_limit:]:
            layer.trainable = False
    my_inception_net_model = tf.keras.Sequential([
        tf.keras.layers.Lambda(lambda image: tf.image.resize(image, (image_size, image_size))),
        inception_net,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(2048, activation='relu'),
        tf.keras.layers.Dropout(0.6),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dropout(0.6),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    adam = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    my_inception_net_model.compile(adam, 'sparse_categorical_crossentropy', metrics=['accuracy'])
    return my_inception_net_model

In [7]:
tuner = kt.Hyperband(
    get_modified_inception_net,
    objective='val_accuracy',
    max_epochs=100,
    factor=3,
    directory='saved_models',
    project_name='inceptionnet_cifar10'
)

87916544/87910968 [==============================] - 0s 0us/step


In [11]:
validation_accuracy_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=7,
)
training_accuracy_callback = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy',
    patience=7,
)
reduce_lr_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.6,
    patience=2,
    verbose=1,
    mode='max',
    min_lr=1e-7
)
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

callbacks = [validation_accuracy_callback, training_accuracy_callback, reduce_lr_on_plateau, ClearTrainingOutput()]

In [12]:
tuner.search(
    train_x,
    train_y,
    validation_data=(test_x, test_y),
    epochs=100,
    callbacks=callbacks
)

Trial 14 Complete [00h 36m 05s]
val_accuracy: 0.8902999758720398

Best val_accuracy So Far: 0.8985999822616577
Total elapsed time: 03h 06m 05s

Search: Running Trial #15

Hyperparameter    |Value             |Best Value So Far 
image_size        |375               |225               
trainable_layer   |10                |10                
learning_rate     |1e-06             |0.0001            
tuner/epochs      |2                 |2                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |4                 |4                 
tuner/round       |0                 |0                 

Epoch 1/2
1563/1563 [==============================] - 964s 613ms/step - loss: 1.9845 - accuracy: 0.3852 - val_loss: 0.5966 - val_accuracy: 0.8069
Epoch 2/2
1563/1563 [==============================] - 956s 612ms/step - loss: 0.8357 - accuracy: 0.7264 - val_loss: 0.4847 - val_accuracy: 0.8384


ResourceExhaustedError: ignored